<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/%E1%84%86%E1%85%B5%E1%84%89%E1%85%A7%E1%86%AB12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
import requests
import zipfile
import io
import os
import unicodedata

# 1. 다운로드할 파일 URL
url = "https://bakey-api.codeit.kr/api/files/resource?root=static&seqId=13987&version=1&directory=/summarization.zip&name=summarization.zip"

# 2. 파일 다운로드
response = requests.get(url)
response.raise_for_status()  # 오류 발생 시 예외 처리

extract_path = "extracted_files"
# 3. 메모리에서 ZIP 파일 열기
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    # 모든 파일을 현재 디렉토리에 풀기
    # z.extractall("extracted_files")
    z.extractall(extract_path)
    # 파일명 리스트 출력
    for info in z.infolist():
        # print("원본 파일명:", info.filename)
        # 파일명 정규화 (NFD → NFC)
        fixed_name = unicodedata.normalize("NFC", info.filename)

        target_ = os.path.join(extract_path, fixed_name)
        with z.open(info) as source, open(target_, "wb") as target:
            target.write(source.read())

        print("추출된 파일:", target_)

# 압축이 풀린 절대경로 출력
print("압축이 풀린 경로:", os.path.abspath(extract_path))
print("압축 해제 완료!")


추출된 파일: extracted_files/summarization/train_original_news.json
추출된 파일: extracted_files/summarization/valid_original_law.json
추출된 파일: extracted_files/summarization/valid_original_editorial.json
추출된 파일: extracted_files/summarization/train_original_law.json
추출된 파일: extracted_files/summarization/valid_original_news.json
추출된 파일: extracted_files/summarization/train_original_editorial.json
압축이 풀린 경로: /content/extracted_files
압축 해제 완료!


In [ ]:
!pip install datasets
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 36.7 MB/s eta 0:00:00


# 데이터 로드 및 전처리 함수 정의

In [ ]:
import os
import json

def load_json_dataset(file_path):
    # 하나의 JSON 파일을 열어 문서별로 text, summary 정보를 추출
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    examples = []
    for doc in data["documents"]:
        sentences = []
        # "text"는 중첩 리스트 형태이므로 내부의 모든 sentence를 추출
        for sublist in doc["text"]:
            for item in sublist:
                sentences.append(item.get("sentence", ""))
        full_text = " ".join(sentences)
        # abstractive 요약은 첫번째 항목 사용 (없으면 빈 문자열)
        summary = doc["abstractive"][0] if doc["abstractive"] else ""
        examples.append({
            "text": full_text,
            "summary": summary,
        })
    return examples

# 기본 경로 설정
# base_path = "/content/drive/Shareddrives/스프린트(AI) 드라이브/트랙 Master 폴더/스프린트 미션 및 모범답안/data/summarization/"
base_path = "extracted_files/summarization/"
# law 관련 파일 하나만 불러오기
train_file = "train_original_editorial.json"
valid_file = "valid_original_editorial.json"

train_examples = load_json_dataset(os.path.join(base_path, train_file))
valid_examples = load_json_dataset(os.path.join(base_path, valid_file))

print("Train examples:", len(train_examples))
print("Validation examples:", len(valid_examples))

Train examples: 56760
Validation examples: 7008


# Hugging Face Dataset 변환 및 전처리

In [ ]:
from transformers import BartTokenizerFast

# Fast Tokenizer 사용
checkpoint = "gogamza/kobart-base-v1"
tokenizer = BartTokenizerFast.from_pretrained(checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


In [ ]:
from datasets import Dataset, DatasetDict
from transformers import BartTokenizer

# train_examples와 valid_examples로 Dataset 객체 생성
train_dataset = Dataset.from_list(train_examples)
valid_dataset = Dataset.from_list(valid_examples)

# DatasetDict 형태로 통합 (훈련/검증)
dataset = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset
})

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import BartTokenizerFast, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)


model_name = "gogamza/kobart-base-v1"
tokenizer = BartTokenizerFast.from_pretrained(model_name)

def tokenize_function(example):
    model_inputs = tokenizer(
        example["text"],
        max_length=512,
        truncation=True
    )
    labels = tokenizer(
        text_target=example["summary"],
        max_length=256,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


Map:   0%|          | 0/56760 [00:00<?, ? examples/s]

Map:   0%|          | 0/7008 [00:00<?, ? examples/s]

# 파인튜닝 설정 및 학습

In [ ]:
from transformers import BartForConditionalGeneration, TrainingArguments, Trainer

model = BartForConditionalGeneration.from_pretrained(checkpoint)
training_args = TrainingArguments(output_dir="test-trainer",
    # T4 최적화 (VRAM 16GB 활용)
    fp16=True,
    per_device_train_batch_size=16,      # 8 → 16 (2배)
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,       # 4 → 2 (실질 배치 = 16 * 2 = 32)

    # 학습 설정
    num_train_epochs=3,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,

    # 로깅/저장
    logging_steps=100,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,

    # T4 추가 최적화
    dataloader_num_workers=2,
    dataloader_pin_memory=True,          # CPU→GPU 전송 빠르게

    report_to="none")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

/tmp/ipython-input-3718464047.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
500,2.241600,2.025534
1000,2.143300,1.948643
1500,2.080800,1.905929
2000,1.867500,1.904799
2500,1.883700,1.881707
3000,1.877700,1.861921
3500,1.865400,1.845362
4000,1.708700,1.856896
4500,1.698100,1.855301
5000,1.702000,1.849485


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=5322, training_loss=1.98402687769463, metrics={'train_runtime': 3793.5028, 'train_samples_per_second': 44.887, 'train_steps_per_second': 1.403, 'total_flos': 5.190947512418304e+16, 'train_loss': 1.98402687769463, 'epoch': 3.0})

# 파인튜닝된 모델로 pipeline 추론 테스트

In [ ]:
from transformers import pipeline

# summarization pipeline 생성 (모델과 토크나이저가 이미 정의되어 있다고 가정)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# 검증 데이터셋의 첫 10개 예시에 대해 요약 결과 출력
for i in range(10):
    sample_text = tokenized_datasets["validation"][i]["text"]
    prompt = f"원문:\n{sample_text}\n요약:"
    print(f"예시 {i+1}:")
    print(prompt)

    # 요약 결과 생성 (필요한 max/min length는 상황에 맞게 조정)
    result = summarizer(prompt, max_length=130, min_length=30, do_sample=False)
    print(result[0]['summary_text'])
    print("-" * 50)

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


예시 1:
원문:
더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 매우 송구하다"고 밝혔다. 더불어민주당 이해찬 대표가 30 일 기자간담회를 열고 '조국 사태'와 관련, "국민 여러분께 매우 송구하다"는 입장을 밝혔다. 이 대표는 "검찰 개혁이란 대의에 집중하다 보니, 국민 특히 청년이 느꼈을 불공정에 대한 상대적 박탈감, 좌절감을 깊이 있게 헤아리지 못했다"며 "여당 대표로서 무거운 책임감을 느낀다"고 머리를 숙였다. 조국 전 법무부 장관이 14 일 사퇴한 이후 이 대표가 당 안팎의 쇄신 요구에 대해 입장을 표명한 것은 이번이 처음이다. 청와대와 여당은 '조국 정국'을 거치며 분출된 '공정'과 '정의'의 민심을 받들어 검찰 개혁에 매진하겠다면서도 두 달간 극심한 분열과 갈등을 초래한데 대해선 진지하게 성찰하는 모습을 보이지 않았다. 그나마 초선인 이철희 의원이 "당이 대통령 뒤에 비겁하게 숨어 있었다"고 비판했고, 표창원 의원은 "책임을 느끼는 분들이 각자 형태로 그 책임감을 행동으로 옮겨야 할 때"라고 지적했다. 뒤늦게나마 이 대표가 자성의 목소리를 내긴 했으나 당 안팎의 쇄신 요구에 어떻게 응할지 구체적 플랜을 제시하지 못해 여전히 안이하다는 지적도 나온다. 이 대표는 28 일 윤호중 사무총장을 단장으로 하는 총선기획단을 발족했고 조만간 인재영입위원회도 출범시킬 계획이라고 밝혔다. 이 대표는 "민주당의 가치를 공유하는 참신한 인물을 영입해 준비된 정책과 인물로 승부하겠다"고 다짐했다. 하지만 당 일각에선 "총선기획단장을 비롯한 당직 인선부터 쇄신 의지를 보여야 한다"는 비판의 목소리가 나온다. 무조건 물러나는 게 능사는 아니지만 국정 혼선을 초래한 데 대해 당 지도부가 겸허하게 책임지는 모습을 보이는 게 쇄신의 출발점이 돼야 한다는 지적도 있다. 선거는 대중의 이해와 요구를 잘 대표하는 정치인을 뽑는 행위다. 민생을 외면하며 낡은 이념과 진영 싸움에 매몰된 구시대 인물들을 과감히 물갈

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


더불어민주당 이해찬 대표가 30일 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 매우 송구하다"는 입장을 밝혔으나 당 안팎의 쇄신 요구에 어떻게 응할지 구체적 플랜을 제시하지 못해 안이하다는 지적도 나오고 있다.     이해찬 대표의 유감 표명이 여권 전반의 인적 쇄신으로 이어지길 기대한다.          국정 혼선을 초래한 데 대해 당 지도부가 겸허하게 책임지는 모습을 보이는 게 쇄신의 출발점이 돼야 한다.                선거는 대중의 이해와 요구를 잘 대표하는 정치인을 뽑는 행위로 민생을 외면하며 낡은 이념과 진영 싸움에 매몰된 구시대 인물들을 과감히 물갈이하라는 국민의 요구와 민생을 외면하며 낡은 이념과 진영 싸움에 매몰된 구시대 인물들을 과감히 물갈이하라는 게 국민의 요구다.            민생을 외면하며 낡은 이념과 진영 싸움에 매몰된 구시대 인물들을 과감히 물갈이하라는 국민의 요구와 민생을 외면하며 낡은 이념과 진영 싸움에 매몰된 구시대 인물들을 과감히 물갈이하라는 국민의 요구는 국민의 요구이다.       
--------------------------------------------------
예시 2:
원문:
탈원전 정책에서 비롯된 한국전력의 경영 부담이 결국 전기료 인상을 초래하게 됐다. 김종갑 한전 사장은 언론 인터뷰에서 "정부 정책에 따라 도입된 각종 전기료 특례할인을 모두 폐지하고 전기요금 원가공개 방안을 정부와 협의하겠다"고 밝혔다. 특례할인은 여름철 누진제, 주택용 절전 및 전기차 충전 등의 명목으로 전기료를 깎아주는 제도로서, 작년 1 조 1434 억원에 달했다. 이 제도가 없어지면 부담은 고스란히 국민에게 돌아가게 된다. 전기료 인상은 예견됐던 것이나 마찬가지다. 문재인 정부 출범 이후 탈원전 정책이 가속화되면서 한전 경영이 악화일로를 달려왔기 때문이다. 원전 가동을 줄이고 발전 단가가 높은 민간 발전사의 전기 구매를 늘린 탓에 한전의 발전 원가는 2016 년 25 조원 수준에서 201

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

더불어민주당 김정호 국회의원이 28일 열린 국회 예산결산특별위원회 전체회의에서 대구공항 통합이전 문제를 들고나온 것에 대해 대구지역 국회의원들은 "국가 정책과 법적 절차에 의해 결정된 신공항 사업에 문제를 제기하는 것은 불순한 의도의 정치 공세"라고 반박했다.     지역 민심과 여론을 한곳으로 모아야 한다.          내우외환에 휘둘려서는 안됩니다.             대구경북 통합신공항 선정 기준을 두고 대구시와 경북도가 최종 절충 방안을 제시한 가운데  연내 이전지 확정이라는 배수진을 치고 있는 상황이다.       군위군수에 대한 경찰의 압수수색과 피의자 신분 조사를 둘러싸고 통합신공항 이전 추진에 악재로 작용하지 않을까 우려하는 목소리도 많다.           내우외환에 휘둘려서는 안된다.        내우외환에 휘둘려서는 안된다.               내우외환에 휘둘려서는 안됩니다.                   
--------------------------------------------------
예시 8:
원문:
이주노동자들에게 적용되는 '출국 후 퇴직금 수령제도'로 인해 이주노동자들이 퇴직금을 받는데 어려움을 겪고 있다. 불법체류를 막기 위해 이주노동자가 출국한 뒤 퇴직금을 받도록 한 '출국만기보험 제도'가 퇴직금 미지급자를 양산하고 있는 것이다. 출국만기보험이란 사업주가 퇴직금을 지급하기 위해 노동자를 피보험자로 해 가입하는 보험이다. 사업주가 매달 통상임금의 8.3%를 보험회사에 적립한 것으로, 노동자가 퇴직 후 출국할 때 공항에서 받을 수 있다. 2014 년 정부는 불법체류 방지를 위해 이주노동자 퇴직금을 '퇴사 후 14 일 이내'에서 '출국 후 14 일 이내' 지급하는 것으로 바꿨다. 하지만 상당수 이주노동자들이 퇴직금 받는 방법을 제대로 모르고 있다. 이주노동자가 퇴직금을 받으려면 구비서류가 많은데다 퇴직금을 수령받는 절차도 복잡하다. 최근 이주노동자를 대상으로한 한 설문조사에 따르면, 출국만기보험금 수령절차를 알고

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


불법체류를 막기 위해 이주노동자가 출국한 뒤 퇴직금을 받도록 한 '출국만기보험 제도'로 인해 이주노동자들이 퇴직금을 받는데 어려움을 겪고 있는 가운데 정부는 조속히 해결책을 마련하여 이주노동자가 더이상 피해를 보지 않도록 해야 한다. @
--------------------------------------------------
예시 9:
원문:
법무부가 제정한 훈령 '형사사건 공개 금지 등에 관한 규정'을 둘러싼 논란이 뜨겁다. 당장 국민의 알권리와 정면 충돌하고 있다는 지적이 비등하고 있다. 법무부의 훈령은 사건 관계인의 인권보호를 위해 검찰의 수사 상황을 전면 비공개하고, 오보 언론에 대한 처벌 수단을 마련하겠다는 게 그 요지다. 구체적으로는 검사와 수사관 등 관계자가 기자와 개별적으로 만날 수 없도록 했다. 또 검찰수사와 관련해 오보를 내면 해당 언론사는 브리핑은 물론 검찰청 출입 자체가 제한된다. 다만 공보 담당자와는 만날 수 있다. 한마디로 기자는 알려주는 내용만 받아쓰라는 것이다. 언론 자유의 침해가 심각하게 우려되는 조치가 아닐 수 없다. 이 규정을 만든 것은 지나친 피의사실 공개로 피의자 인권이 침해당하는 폐해를 막기 위한 조치라는 게 법무부 설명이다. 전혀 틀린 얘기는 아니다. 실제 그동안 검찰이 피의사실을 흘려 망신을 주고, 사실상 여론재판이 이뤄지는 바람에 공정한 재판을 받을 피의자의 권리가 심각하게 훼손되는 경우가 다반사였다. 노무현 전 대통령에 대한 '논두렁 시계' 사건 등이 대표적 사례다. 특히 삼성 등 기업 관련 수사는 미확인 내용이 사실처럼 무차별 유포되면서 기업의 국제신인도가 추락하는 부작용을 낳기도 했다. 이런 폐단을 검찰 개혁 차원에서 없자는 취지는 백번 공감이 간다. 하지만 그 제한이 과도하다는 게 문제다. 당초 취지와도 맞지 않는다. 무엇보다 기자의 브리핑 참석과 출입 제한은 받아들이기 어려운 사안이다. 정보 접근이 어렵거나 취재원의 한정과 시간상 제약 등으로 의도치 않은 '오보'는 보도는 과정에서 필연적일 수밖에 없다. 

Both `max_new_tokens` (=256) and `max_length`(=130) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


법무부가 훈령은 사건 관계인의 인권보호를 위해 검찰의 수사 상황을 전면 비공개하고, 오보 언론에 대한 처벌 수단을 마련하겠다는 게 그 요지인데 이 훈령은 사건 관계인의 인권보호를 위해 검찰의 수사 상황을 전면 비공개하고, 오보 언론에 대한 처벌 수단을 마련하겠다는 게 그 요지다.  정부가 특정 사건을 은폐 축소하더라도 이를 견제할 방법도 없다.  정부가 규정 시행을 밀어붙인다면 비판 언론에 재잘을 물리려는 의도로 밖에 볼 수 밖에 없다.   국가가 언론을 통제하고 심지어 강제 통폐합을 하는 만행도 저질렀다.      군사정권 시절에도 없었던 군사정권 시절에도 없었던      군사정권 시절에도 없었던  군사정권의    군사정권의  과거 적폐가 되살아 날 수도 있다.           정부는 언론 자유의 침해를 막아야 한다.         정부가 규정 시행을 밀어붙인다면 비판 언론에 재잘을 물리려는 의도로 밖에 볼 수 없다.         정부가 규정 시행을 밀어붙인다면 비판 언론에 재잘을 물리려는 의도로 밖에 볼 수 없다.                 
--------------------------------------------------
예시 10:
원문:
딱 한달이다. 두달도 이어가지 못했다. 5 개월만에 생산·소비·투자 등 경제 활동 3 대 지표가 모두 상승하며 희망을 줬던 산업활동동향(8 월)이 다시 한숨 속 위기감으로 빠져드는데는 한달이면 충분했다. 통계청이 31 일 발표한 '9 월 산업활동 동향'에서 지난달 전(全)산업 생산지수(계절조정계열)는 108.0 으로, 전월보다 0.4% 감소했다. 광공업 생산은 증가했지만, 도소매와 금융·보험업을 중심으로 서비스업 생산이 줄어든 것이 영향을 미쳤다. 소매판매는 음식료품과 의복 판매가 감소하면서 전월보다 2.2%줄어들었다. 감소폭은 2017 년 12 월(-2.4%) 이후 가장 컸다. 이제는 익숙해져 그러려니 하지만 통계청의 설명은 여지없이 기저효과와 날씨 때문이란 설명으로 일관한다. 올해 추석이 이른 9 월

In [ ]:
# 1. 필요한 라이브러리 설치 및 import
!pip install evaluate rouge_score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
import evaluate
import torch
from tqdm import tqdm

# 1. 채점 도구 가져오기
rouge = evaluate.load('rouge')

# 2. 채점할 문제 개수 (너무 많으면 오래 걸리니 5개만!)
num_samples = 5
samples = valid_dataset.shuffle(seed=42).select(range(num_samples))

predictions = [] # 로봇이 쓴 답
references = []  # 정답지

print("채점을 시작합니다...")

# 3. 문제 풀기
model.eval() # 평가 모드로 전환
for batch in tqdm(samples):
    input_text = batch['text'] # 원문
    target_text = batch['summary'] # 정답 요약

    # 로봇에게 요약 시키기
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=128,
            num_beams=3,
            early_stopping=True
        )

    # 로봇의 답안지 작성
    pred = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    predictions.append(pred)
    references.append(target_text)

    # 눈으로 확인해보기
    print(f"\n[원문] {input_text[:50]}...")
    print(f"[정답] {target_text}")
    print(f"[로봇] {pred}")

# 4. 점수 계산하기
results = rouge.compute(predictions=predictions, references=references)

print("\n" + "="*30)
print(f"성적표 (ROUGE 점수)")
print("="*30)
print(f"ROUGE-1 (단어 일치도): {results['rouge1']:.4f}")
print(f"ROUGE-2 (연속 단어 일치도): {results['rouge2']:.4f}")
print(f"ROUGE-L (문장 구조 일치도): {results['rougeL']:.4f}")

채점을 시작합니다...


 20%|██        | 1/5 [00:01<00:05,  1.27s/it]


[원문] 상장법인들이 금요일에 주총을 몰아서 하는 구태가 올해도 어김없이 되풀이되고 있어 참으로 한...
[정답] 금요일에 다 같이 주총을 몰아 하는 것은 주주들이 참석하지 못하도록 하는 기업의 속셈이기 때문에, 국회는 주주들이 주총에 참가하지 않아도 전자적으로 의결권을 행사할 수 있는 전자투표제를 활성화해서 몰아서 주총을 여는 기업의 메리트를 없애야 한다.
[로봇] 상장법인들이 금요일에 주총을 몰아서 하는 구태가 올해도 어김없이 되풀이되고 있어 참으로 한심하며 주총에 직접 참가하지 않고도 전자적 방법으로 의결권을 행사할 수 있는 전자투표제를 활성화시킴으로써 몰아치기 주총의 메리트를 없애는 것도 한 방법이다.   주주들이 기업 비전을 공유하고 건전한 지배구조를 갖춰 기업 경영의 투명성을 높이기 위해서도 다수 주주들이 참석할 수 있도록 주주들의 참가 편의를 우선해야 마땅하다.    주총을 몰아서 여는 데는 뭔가 켕기는 게 있을 것이라는 의심을 살 수밖에 없다.  사실 주총 일정까지 강제로


 40%|████      | 2/5 [00:02<00:03,  1.25s/it]


[원문] 중국의 인권 신장을 위해 비폭력 투쟁을 벌여온 류샤오보(劉曉波ㆍ55)가 올해 노벨평화상 수...
[정답] 지구촌의 보편적 가치인 민주주의와 인권을 위해 용기 있게 싸워온 양심적인 지식인으로  중국의 인권 신장을 위해 비폭력 투쟁을 벌여온 류샤오보를  올해 노벨평화상 수상자로 선정하고 중국 정부에게 자국민의 정치적 권리와 인권 신장에 대한 더 큰 책임을 요구하였다.
[로봇] 중국의 인권 신장을 위해 비폭력 투쟁을 벌여온 류샤오보(劉曉波ᆞ55)가 올해 노벨평화상 수상자로 선정됐는데 중국의 인권 신장을 위해 비폭력 투쟁을 벌여온 양심적인 지식인에게 최고의 경의와 격려를 보내며 이번 노벨평화상 결정도 서방의 음모나 내정간섭으로만 치부할 게 아니라 스스로 긍정적인 변화를 만들어가는 계기로 삼기 바란다.     자유와 인권을 존중하면서 '평화롭게 서는 서(和起)' 중국의 미래를 기대한다.       중국의 미래를 기대한다.      


...


[원문] 부산 대표 도서관을 목표로 수백억 원의 예산이 투입된 부산도서관이 미세먼지 날리는 시멘트 ...
[정답] 부산시는 도시 발전과 주민 건강을 위해 미세먼지를 유발하는 시멘트 공장을 이전하고 서부산 균형 발전을 견인할 대표 도서관 설립에 총력을 다해야 한다. 
[로봇] 부산도서관이 미세먼지 날리는 시멘트 공장 옆에서 개관하게 생겼는데 부산시는 서부산 균형발전을 견인할 부산 대표 도서관을 만드는 일에 만전을 기해야 하며 서부산 균형발전을 견인할 부산 대표 도서관을 만드는 일에 만전을 기해야 한다.         부산도서관은 한국의 빌 게이츠를 키울 문화와 교육의 중심이 되어야 한다.               부산도서관은 한국의 빌 게이츠를 키울 문화와 교육의 중심이 되어야 한다.                      


 80%|████████  | 4/5 [00:04<00:01,  1.24s/it]


[원문] 이탈리아·미국계 자동차 업체인 피아트크라이슬러(FCA)와 프랑스 푸조시트로엥(PSA)그룹이...
[정답] 글로벌 자동차 업체들이 생존을 위해 절박하게 합종연횡을 진행하는 가운데 이탈리아·미국계 자동차 업체인 피아트크라이슬러(FCA)와 프랑스 푸조시트로엥(PSA)그룹이 합병에 합의하면서, 우리도 이러한 혁명적인 변화에 대응하기 위해서는 노사정의 일대 각성과 발 빠른 대응이 필요한 시점이다.
[로봇] 피아트크라이슬러(FCA)와 프랑스 푸조시트로엥(PSA)그룹이 합병에 합의해 세계 4위 자동차 기업이 된 가운데 한국 자동차 산업이 미래 강자로 거듭나기 위해서는 노사 협력으로 고비용·저생산성 구조를 깨는 일도 시급하다.    한국 자동차가 미래 강자로 거듭나기 위해서는 제도 정비와 생태계 구축을 서둘러야 한다.     한국 자동차가 미래 강자로 거듭나기 위해서는 한국 자동차가 미래 강자로 거듭나야 한다.                        


100%|██████████| 5/5 [00:06<00:00,  1.24s/it]


[원문] 한일 군사정보보호협정(GSOMIA·지소미아)이 조건부로 연장됐다. 지소미아 종료를 몇 시간...
[정답] 지소미야 연장을 강력히 요구해온 미국과 일본은 환영이겠으나 국내 사정은 엇갈리며 한일 양국의 움직임이 빨라지고 있는 것은 그나마 다행이며, 조건부 연장 결정은 일본 측의 물밑으로 접촉해온 결과다.
[로봇] 한일 군사정보보호협정(GSOMIA·지소미아)이 조건부로 연장되었지만 한일 무역갈등을 불러일으킨 강제징용 배상은 꺼지지 않는 불씨이기에 한일 양국의 화해 무드에 더없이 좋은 징조가 될 것이다.                                                                          

성적표 (ROUGE 점수)
ROUGE-1 (단어 일치도): 0.1600
ROUGE-2 (연속 단어 일치도): 0.1333
ROUGE-L (문장 구조 일치도): 0.1600
